<a href="https://colab.research.google.com/github/park-hoyeon/park-hoyeon.github.io/blob/master/skt_7_02_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 전처리

- 토큰화: 실제로	사용하는	문장들을	모아서	만든	텍스트	데이터를	말뭉치(Corpus,	코퍼스)라고	한다.
말뭉치의	문장들을	문법적인	최소	단위(Token,	토큰)로	나누는	작업을	토큰화(Tokenization)
라고	한다.

In [ ]:
sentences  = [
 'I love my dog',
 'I love my cat',
 'You love my dog!',
 'Do you think my dog is amazing?'
]

num_words는	출현	빈도에	따라서	저장할	최대	단어	개수를	설정한다.<br>
oov_token는	최대	저장	단어의	개수를	넘어간	단어를	표시할	문자열을	지정한다.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 100, oov_token = '<OOV>')
tokenizer.fit_on_texts(sentences)

In [ ]:
word_index = tokenizer.word_index
print(word_index)

In [ ]:
# 문자열인 문장을 숫자로 변환한다.
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

In [ ]:
# 패딩 설정 - 최대 시퀀스의 길이보다 짧은 시퀀스는 나머지를 0으로 채운다.
from tensorflow.keras.preprocessing.sequence import pad_sequences
sentences = pad_sequences(sequences, maxlen=6, padding='pre')
print(sentences)

In [ ]:
# 인코딩
from tensorflow.keras.utils import to_categorical
sequences = to_categorical(sentences)
print(sequences[0])
print(sequences.shape)

In [ ]:
# 임베딩 레이어 사용
# 인공 신경망에서는 임베딩 벡터들을 학습을 통해 자동으로 생성한다. - Keras의 Embeding 레이어를 확인한다.
from tensorflow.keras.layers import Embedding
embedding = Embedding(input_dim=100, output_dim=3, input_length=6)
output = embedding(sentences)
print(output)

# IMDB의 긍정/부정 분류
IMDB	데이터는	영화	리뷰에	대한	데이터이다.	IMDB	데이터를	사용해서	리뷰가	긍정인지	부정
인지를	분류하는	과정을	확인한다.

In [ ]:
# 데이터 준비
from keras.datasets import imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=500)
print(X_train.shape, X_test.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# 데이터 확인
print(X_train[0])
print('length : ', len(X_train[0]))

In [ ]:
# 타깃값을 출력해서 확인한다. - 1은 긍정, 0은 부정
print(y_train[0])


In [ ]:
# 문장 복원 - 데이터의 첫 번째 문장을 숫자에서 문자열로 복원해서 확인한다.
# 먼저 word index를 출력해서 확인하기

print(imdb.get_word_index())

In [ ]:
# word index를 역으로 숫자를 키로 가지는 사전을 만든다.
word_index = imdb.get_word_index()
reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()]
)
print(reverse_word_index)

In [ ]:
# 숫자로 인코딩된 문장을 문자로 바꾼다.
# 0,1,2 는 특수한 목적으로 사용하므로 단어를 할당하지 않고 ?로 설정한다.

decode_review = ' '.join(
    [reverse_word_index.get(i-3, '?') for i in X_train[0]]
)
print(decode_review)

### 시퀀스 만들기
- 입력으로 사용하기 위해서는 모든 리뷰가 같은 길이의 시퀀스 데이터이어야 한다,
- 적당한 시퀀스의 길이를 확정하기 위한 절차를 진행한다.
- 모든 문장의 길이를 조사한다.


In [ ]:
import numpy as np
lengths = np.array([len(x) for x in X_train])
print(lengths[:20])

In [ ]:
# 문장 길이의 분포를 확인한다.
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 7))
plt.hist(lengths)
plt.xlabel('length')
plt.ylabel('frequency')
plt.show()

In [ ]:
# 시퀀스의 최대 길이를 200으로 설정하고 시퀀스를 만든다.
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 200
X_train = pad_sequences(X_train, maxlen=max_len)
print(X_train.shape)

In [ ]:
# 시퀀스 데이터를 몇 개만 출력해서 확인한다.
print(X_train[:2])

In [ ]:
# 원-핫 인코딩
# 함수의 결과는 Tensor 타입이다. 출력되는 Shpae에 주목한다.
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

train_oh = keras.utils.to_categorical(X_train)
print(train_oh.shape)

In [ ]:
# 모델 만들기 - LSTM 레이어를 사용하며, 입력 데이터의 형태와 출력의 활성화 함수에 주의한다.
model = keras.Sequential()
model.add(keras.layers.Embedding(500, 16, input_length=200))
model.add(keras.layers.LSTM(64, return_sequences=True))
model.add(keras.layers.LSTM(32))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.summary()